## Cleaning Strings

we'll be working with the restaurants DataFrame which has data on various restaurants. Our ultimate goal is to create a restaurant recommendation engine, but we need to first clean the data.

In [4]:
# Import pandas
import pandas as pd

# Import process from fuzzywuzzy
from fuzzywuzzy import process

In [8]:
# Read csv
data=pd.read_csv("datasets//restaurants_L2.csv")

# Info of csv
print(data.info())

# Let's see have a look in the data
print(data.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 336 entries, 0 to 335
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  336 non-null    int64 
 1   name        336 non-null    object
 2   addr        336 non-null    object
 3   city        336 non-null    object
 4   phone       336 non-null    int64 
 5   type        336 non-null    object
dtypes: int64(2), object(4)
memory usage: 15.9+ KB
None
   Unnamed: 0                       name                       addr  \
0           0  arnie morton's of chicago   435 s. la cienega blv .    
1           1         art's delicatessen       12224 ventura blvd.    
2           2                  campanile       624 s. la brea ave.    
3           3                      fenix    8358 sunset blvd. west    
4           4         grill on the alley           9560 dayton way    

          city       phone      type  
0  los angeles  3102461501  american  
1  studio city 

In [9]:
# Store the unique values of cuisine_type in unique_types
unique_types = data["type"].unique()

# print the unique values
print(unique_types)

['american' 'asian' 'italian' 'coffeebar' 'mexican' 'southwestern'
 'steakhouses' 'southern' 'cajun']
